In [1]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [ ]:
# データのパス
data_path = 'data/processed/android_cleaned_mecab.csv'

# データの読み込み
df = pd.read_csv(data_path)

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

In [3]:
# データのnegative_review_keywordsに対してだけ，Topic Modelingを行う

negative_review_keywords = df['negative_review_keywords'].dropna().tolist()

In [4]:
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# umapとhdbscanのモデルを作成
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)

# BERTopicモデルを作成

topic_model = BERTopic(
    language="japanese",
    calculate_probabilities=False,
    verbose=True,
    nr_topics="20",
    umap_model=umap_model, 
    hdbscan_model=hdbscan_model
    )

topics, probs = topic_model.fit_transform(negative_review_keywords)

In [21]:
# モデルをsafetensorsで保存する
topic_model.save('models/android_topic_model', serialization="safetensors", save_embedding_model=True, save_ctfidf=True)

In [ ]:
topic_model.get_topic_info()

In [20]:
# トピックIDと名前の対応付け
topic_mapping = topic_model.get_topic_info().set_index('Topic')['Name'].to_dict()

# topics と probs はそれぞれリストまたは NumPy 配列と仮定
# negative_review_keywords の行数と topics, probs の長さが一致している前提
df['topic'] = topics  # トピックIDを新しい列 'topic' として追加
df['topic_probability'] = probs  # トピック確率を新しい列 'topic_probability' として追加
df['topic_name'] = df['topic'].map(topic_mapping)  # トピックIDをトピック名に変換

# dfを保存
df.to_csv('data/processed/android_cleaned_mecab_with_topics.csv', index=False)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_barchart(
    top_n_topics=14,
    width=500,  # 幅を広げる
)

In [ ]:
# topics, probs = topic_model.fit_transform(negative_review_keywords)の結果を元のデータに新たなカラムとして追加

df['topic'] = topics

df